In [1]:
import pyspark
from delta import *

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

Py4JJavaError: An error occurred while calling o46.saveAsTable.
: org.apache.spark.SparkException: Unable to create database default as failed to create its directory file:/C:/Users/levy/Desktop/proj/spk/spk/spark-warehouse.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unableToCreateDatabaseAsFailedToCreateDirectoryError(QueryExecutionErrors.scala:539)
	at org.apache.spark.sql.catalyst.catalog.InMemoryCatalog.liftedTree1$1(InMemoryCatalog.scala:126)
	at org.apache.spark.sql.catalyst.catalog.InMemoryCatalog.createDatabase(InMemoryCatalog.scala:119)
	at org.apache.spark.sql.internal.SharedState.externalCatalog$lzycompute(SharedState.scala:159)
	at org.apache.spark.sql.internal.SharedState.externalCatalog(SharedState.scala:140)
	at org.apache.spark.sql.internal.BaseSessionStateBuilder.$anonfun$catalog$1(BaseSessionStateBuilder.scala:154)
	at org.apache.spark.sql.catalyst.catalog.SessionCatalog.externalCatalog$lzycompute(SessionCatalog.scala:122)
	at org.apache.spark.sql.c

In [4]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("../data/country_ISO_3166.csv")
)

In [5]:
df.show()

+-------------------+---------------------+---------------------+--------------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|       country_name|country_code_2_letter|country_code_3_letter|country_code_numeric|   iso_3166_2|  region|          sub_region|intermediate_region|region_code|sub_region_code|intermediate_region_code|
+-------------------+---------------------+---------------------+--------------------+-------------+--------+--------------------+-------------------+-----------+---------------+------------------------+
|        Afghanistan|                   AF|                  AFG|                   4|ISO 3166-2:AF|    Asia|       Southern Asia|               NULL|        142|             34|                    NULL|
|      Åland Islands|                   AX|                  ALA|                 248|ISO 3166-2:AX|  Europe|     Northern Europe|               NULL|        150|            154|      

In [7]:
df.write.format("delta").save("country")

Py4JJavaError: An error occurred while calling o112.save.
: ExitCodeException exitCode=-1073741515: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.Chec